In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

In [2]:
X_train = pd.read_csv("../data/X_train.csv")
X_val   = pd.read_csv("../data/X_val.csv")
X_test  = pd.read_csv("../data/X_test.csv")

y_train = pd.read_csv("../data/y_train.csv").squeeze()
y_val   = pd.read_csv("../data/y_val.csv").squeeze()
y_test  = pd.read_csv("../data/y_test.csv").squeeze()

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("Train class balance:", y_train.value_counts(normalize=True).to_dict())

X_train: (25519, 10) y_train: (25519,)
Train class balance: {0: 0.9894980210823308, 1: 0.010501978917669188}


In [3]:
numeric_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = X_train.select_dtypes(exclude=["number"]).columns.tolist()

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

Numeric columns: ['AMT_INCOME_TOTAL', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'CNT_FAM_MEMBERS', 'FLAG_MOBIL']
Categorical columns: ['OCCUPATION_TYPE', 'CODE_GENDER', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE']


In [4]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))  # sparse by default
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols),
    ],
    remainder="drop"
)

In [5]:
def evaluate(model, X, y, name="SET"):
    pred = model.predict(X)
    return {
        "set": name,
        "accuracy": accuracy_score(y, pred),
        "precision": precision_score(y, pred, zero_division=0),
        "recall": recall_score(y, pred, zero_division=0),
        "f1": f1_score(y, pred, zero_division=0),
        "confusion_matrix": confusion_matrix(y, pred),
        "report": classification_report(y, pred, zero_division=0),
    }

def print_eval(res):
    print(f"\n=== {res['set']} ===")
    print("Accuracy :", res["accuracy"])
    print("Precision:", res["precision"])
    print("Recall   :", res["recall"])
    print("F1       :", res["f1"])
    print("Confusion matrix:\n", res["confusion_matrix"])
    print("\nReport:\n", res["report"])

In [6]:
svm_v1 = Pipeline(steps=[
    ("preproc", preprocessor),
    ("clf", SVC(kernel="linear", C=1.0))  # no class_weight yet
])

svm_v1.fit(X_train, y_train)

res_train_v1 = evaluate(svm_v1, X_train, y_train, "TRAIN (v1)")
res_val_v1   = evaluate(svm_v1, X_val, y_val, "VAL (v1)")
res_test_v1  = evaluate(svm_v1, X_test, y_test, "TEST (v1)")

print_eval(res_train_v1)
print_eval(res_val_v1)
print_eval(res_test_v1)


=== TRAIN (v1) ===
Accuracy : 0.9894980210823308
Precision: 0.0
Recall   : 0.0
F1       : 0.0
Confusion matrix:
 [[25251     0]
 [  268     0]]

Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     25251
           1       0.00      0.00      0.00       268

    accuracy                           0.99     25519
   macro avg       0.49      0.50      0.50     25519
weighted avg       0.98      0.99      0.98     25519


=== VAL (v1) ===
Accuracy : 0.989394770524776
Precision: 0.0
Recall   : 0.0
F1       : 0.0
Confusion matrix:
 [[5411    0]
 [  58    0]]

Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      5411
           1       0.00      0.00      0.00        58

    accuracy                           0.99      5469
   macro avg       0.49      0.50      0.50      5469
weighted avg       0.98      0.99      0.98      5469


=== TEST (v1) ===
Accuracy : 0.98957761930

In [7]:
svm_v2 = Pipeline(steps=[
    ("preproc", preprocessor),
    ("clf", SVC(kernel="linear", C=1.0, class_weight="balanced", max_iter=15000, tol=1e-3))
])

svm_v2.fit(X_train, y_train)

res_train_v2 = evaluate(svm_v2, X_train, y_train, "TRAIN (v2)")
res_val_v2   = evaluate(svm_v2, X_val, y_val, "VAL (v2)")
res_test_v2  = evaluate(svm_v2, X_test, y_test, "TEST (v2)")

print_eval(res_train_v2)
print_eval(res_val_v2)
print_eval(res_test_v2)

C:\Users\nazza\anaconda3\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=15000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(



=== TRAIN (v2) ===
Accuracy : 0.2363337121360555
Precision: 0.010792099368764
Recall   : 0.7910447761194029
F1       : 0.02129369224588188
Confusion matrix:
 [[ 5819 19432]
 [   56   212]]

Report:
               precision    recall  f1-score   support

           0       0.99      0.23      0.37     25251
           1       0.01      0.79      0.02       268

    accuracy                           0.24     25519
   macro avg       0.50      0.51      0.20     25519
weighted avg       0.98      0.24      0.37     25519


=== VAL (v2) ===
Accuracy : 0.2316694093984275
Precision: 0.010396975425330813
Recall   : 0.7586206896551724
F1       : 0.020512820512820513
Confusion matrix:
 [[1223 4188]
 [  14   44]]

Report:
               precision    recall  f1-score   support

           0       0.99      0.23      0.37      5411
           1       0.01      0.76      0.02        58

    accuracy                           0.23      5469
   macro avg       0.50      0.49      0.19      5469
wei

In [14]:
C_values = [0.001, 0.01, 0.1, 1.0,2.0,3.0,7.0, 10.0,55.0, 100.0]
rows = []

for C in C_values:
    model = Pipeline(steps=[
        ("preproc", preprocessor),
        ("clf", LinearSVC(C=C, class_weight="balanced", random_state=42, max_iter=20000))
    ])
    model.fit(X_train, y_train)
    pred_val = model.predict(X_val)

    rows.append({
        "C": C,
        "val_precision": precision_score(y_val, pred_val, zero_division=0),
        "val_recall": recall_score(y_val, pred_val, zero_division=0),
        "val_f1": f1_score(y_val, pred_val, zero_division=0),
    })

results_C = pd.DataFrame(rows).sort_values("val_f1", ascending=False)
results_C


,C,val_precision,val_recall,val_f1
2,0.100,0.010347,0.344828,0.020090
3,1.000,0.010320,0.344828,0.020040
4,2.000,0.010320,0.344828,0.020040
5,3.000,0.010320,0.344828,0.020040
6,7.000,0.010309,0.344828,0.020020
1,0.010,0.010309,0.344828,0.020020
8,55.000,0.010309,0.344828,0.020020
7,10.000,0.010309,0.344828,0.020020
9,100.000,0.010309,0.344828,0.020020
0,0.001,0.010111,0.344828,0.019646


In [16]:
best_C = results_C.iloc[0]["C"]
print("Best C by validation F1:", best_C)

X_train_full = pd.concat([X_train, X_val], axis=0)
y_train_full = pd.concat([y_train, y_val], axis=0)

svm_v3 = Pipeline(steps=[
    ("preproc", preprocessor),
    ("clf", LinearSVC(C=float(best_C), class_weight="balanced", random_state=42, max_iter=20000))
])

svm_v3.fit(X_train_full, y_train_full)

res_test_v3 = evaluate(svm_v3, X_test, y_test, "TEST (v3 LinearSVC tuned C)")

print_eval(res_test_v3)

Best C by validation F1: 0.1

=== TEST (v3 LinearSVC tuned C) ===
Accuracy : 0.631194002559883
Precision: 0.015810276679841896
Recall   : 0.5614035087719298
F1       : 0.030754444978375782
Confusion matrix:
 [[3420 1992]
 [  25   32]]

Report:
               precision    recall  f1-score   support

           0       0.99      0.63      0.77      5412
           1       0.02      0.56      0.03        57

    accuracy                           0.63      5469
   macro avg       0.50      0.60      0.40      5469
weighted avg       0.98      0.63      0.76      5469



In [20]:
C_values = [0.1, 1, 10]
gamma_values = ["scale", 0.1, 0.01]

rows = []
for C in C_values:
    for gamma in gamma_values:
        model = Pipeline(steps=[
            ("preproc", preprocessor),
            ("clf", SVC(kernel="rbf", C=C, gamma=gamma, class_weight="balanced"))
        ])
        model.fit(X_train, y_train)
        pred_val = model.predict(X_val)

        rows.append({
            "C": C,
            "gamma": gamma,
            "val_precision": precision_score(y_val, pred_val, zero_division=0),
            "val_recall": recall_score(y_val, pred_val, zero_division=0),
            "val_f1": f1_score(y_val, pred_val, zero_division=0),
            "val_pred_pos": int((pred_val == 1).sum())
        })

rbf_results = pd.DataFrame(rows).sort_values("val_f1", ascending=False)
rbf_results

,C,gamma,val_precision,val_recall,val_f1,val_pred_pos
6,10.0,scale,0.026316,0.448276,0.049713,988
7,10.0,0.1,0.025617,0.465517,0.048561,1054
3,1.0,scale,0.021143,0.465517,0.040449,1277
4,1.0,0.1,0.019708,0.465517,0.037815,1370
0,0.1,scale,0.018433,0.413793,0.035294,1302
1,0.1,0.1,0.016643,0.396552,0.031944,1382
8,10.0,0.01,0.016129,0.413793,0.031048,1488
2,0.1,0.01,0.010671,0.362069,0.020731,1968
5,1.0,0.01,0.010547,0.275862,0.020317,1517


In [21]:
best = rbf_results.iloc[0]
best_C = float(best["C"])
best_gamma = best["gamma"]

print("Best RBF params by validation F1:", best_C, best_gamma)

X_train_full = pd.concat([X_train, X_val], axis=0)
y_train_full = pd.concat([y_train, y_val], axis=0)

svm_v4 = Pipeline(steps=[
    ("preproc", preprocessor),
    ("clf", SVC(kernel="rbf", C=best_C, gamma=best_gamma, class_weight="balanced"))
])

svm_v4.fit(X_train_full, y_train_full)

res_test_v4 = evaluate(svm_v4, X_test, y_test, f"TEST (v4 RBF, C={best_C}, gamma={best_gamma})")
print_eval(res_test_v4)


Best RBF params by validation F1: 10.0 scale

=== TEST (v4 RBF, C=10.0, gamma=scale) ===
Accuracy : 0.7957579082099104
Precision: 0.027629233511586453
Recall   : 0.543859649122807
F1       : 0.05258693808312129
Confusion matrix:
 [[4321 1091]
 [  26   31]]

Report:
               precision    recall  f1-score   support

           0       0.99      0.80      0.89      5412
           1       0.03      0.54      0.05        57

    accuracy                           0.80      5469
   macro avg       0.51      0.67      0.47      5469
weighted avg       0.98      0.80      0.88      5469

